[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=ShowOwnOrders.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=ShowOwnOrders.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Show Own Orders

This code shows how to retrieve only your own orders on a market using the `load_my_orders()` method.

This code will place two orders on SOL-PERP, wait for the transactions to confirm, then show all open orders, then just open orders from the current `Account`, then cancel those two orders.

In [ ]:
import decimal
import mango

# Use our hard-coded devnet wallet for DeekipCw5jz7UgQbtUbHQckTYGKXWaPQV4xY93DaiM6h.
# For real-world use you'd load the bytes from the environment or a file.
wallet = mango.Wallet(bytes([67,218,68,118,140,171,228,222,8,29,48,61,255,114,49,226,239,89,151,110,29,136,149,118,97,189,163,8,23,88,246,35,187,241,107,226,47,155,40,162,3,222,98,203,176,230,34,49,45,8,253,77,136,241,34,4,80,227,234,174,103,11,124,146]))

# Create a 'devnet' Context
with mango.ContextBuilder.build(cluster_name="devnet") as context:
    # Load the user's account
    group = mango.Group.load(context)
    accounts = mango.Account.load_all_for_owner(context, wallet.address, group)
    account = accounts[0]

    market_operations = mango.operations(context, wallet, account, "SOL-PERP", dry_run=False)

    print("Orders (initial):")
    print(market_operations.load_orderbook())

    # Go on - try to buy 1 SOL for $10.
    order1 = mango.Order.from_values(side=mango.Side.BUY,
                                     price=decimal.Decimal(10),
                                     quantity=decimal.Decimal(1),
                                     order_type=mango.OrderType.POST_ONLY)
    placed_order1 = market_operations.place_order(order1)
    print("\n\nplaced_order1", placed_order1)

    # And $11.
    order2 = mango.Order.from_values(side=mango.Side.BUY,
                                     price=decimal.Decimal(11),
                                     quantity=decimal.Decimal(1),
                                     order_type=mango.OrderType.POST_ONLY)
    placed_order2 = market_operations.place_order(order2)
    print("\n\nplaced_order2", placed_order2)

    print("Waiting for place order transactions to confirm...\n")
    mango.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, [*placed_order1, *placed_order2]
        )

    print("\n\nOrders (including our new orders):")
    print(market_operations.load_orderbook())

    print("\n\nOur Own Orders:")
    cancellaton_signatures = []
    for order in market_operations.load_my_orders():
        cancellaton_signature = market_operations.cancel_order(order)
        print(cancellaton_signature)
        cancellaton_signatures += cancellaton_signature

    print("Waiting for cancellation transactions to confirm...\n")
    mango.WebSocketTransactionMonitor.wait_for_all(
            context.client.cluster_ws_url, cancellaton_signatures, commitment="processed"
        )

    print("\n\nOrders (without our orders):")
    print(market_operations.load_orderbook())

print("Example complete.")